https://www.quora.com/Which-is-the-best-way-to-measure-deep-autoencoder-performance    

https://stackoverflow.com/questions/48775305/what-function-defines-accuracy-in-keras-when-the-loss-is-mean-squared-error-mse

https://stackoverflow.com/questions/56519500/what-does-accuracy-metrics-mean-in-keras-sample-denoising-autoencoder

In [ ]:
from keras.models import Model
from keras.layers import Input
from ae_model import encoder_vgg16, decoder1_vgg16, decoder2_vgg16, decoder3_vgg16, encoder1_vgg16

input_shape = Input(shape=[256, 192, 3])
ae = Model(input_shape, decoder1_vgg16(encoder_vgg16(input_shape)))
ae.summary()

In [ ]:
import os
import cv2
import numpy as np
from tqdm import tqdm
from keras.preprocessing.image import img_to_array, load_img

datasets = []
path = './grid-search/dataset-bc/JPEGImages/'

files = os.listdir(path)


for i in tqdm(files):
    img = load_img(path+i, target_size=(192, 256))
    datasets.append(img_to_array(img))

data_array = np.reshape(datasets, (len(datasets), 256, 192, 3))
data_array = data_array.astype('float32') / 255.

print(img.size)
print(data_array.shape)

In [ ]:
#split dataset
from sklearn.model_selection import train_test_split

x_train, x_test = train_test_split(data_array, test_size=0.2)


### Tune the batch sizes and number of epochs   

in this grid search phase, i didn't save and weights of model.     



In [ ]:
from keras.callbacks import TensorBoard
import tensorflow as tf
from datetime import datetime
%load_ext tensorboard


tf.random.set_seed(7)


# batch_size = [8, 16 ,32]
# epoch = [50, 100]
# first time test

#sec.time test for more epochs with 32 batch sizes
# batch_size = [32, 40] # 40 is the largest number for Blood Cell dataset and gtx 1080ti. Largger number will call out of memory.
# epoch = [100, 200, 300]

# not very well. Need test datasets  ..... 

# But only test 100 epochs and 200 epochs with 32 or 40 batch sizes...   
batch_size = [32, 40]
epoch = [100, 200]

for bb in batch_size:
    for ee in epoch:

        # log_dir = './grid-search/logs/find_batchs_and_epochs/' + str(bb) + '_' + str(ee) + 'epochs'
        log_dir = './grid-search/logs/find_batchs_and_epochs_with_tests/' + str(bb) + '_' + str(ee) + 'epochs'
        print('logs save in:', log_dir)

        work_text = str(bb) + 'batch_sizes___' + str(ee) + 'epochs'
        work_time = datetime.now().strftime("%Y%m%d-%H%M%S")
        work_config = 'Adam(Default Config)_MSE(Default Config)_Accuracy(mereics)'
        dataset_config = 'train:' + str(x_train.shape) + ';;; test:' + str(x_test.shape)
        fw =  tf.summary.create_file_writer(log_dir)
        with fw.as_default():
            tf.summary.text("Test Title", work_text, step=0)
            tf.summary.text("Time", work_time, step=0)
            tf.summary.text("Config", work_config, step=0)
            tf.summary.text("Datasets_config", dataset_config, step=0)

        ae.compile(optimizer='adam', loss='mean_squared_error', metrics=['accuracy'])
        # ae.fit(data_array, data_array, epochs=ee, batch_size=bb, shuffle=True, verbose=1, callbacks=[TensorBoard(log_dir=log_dir)])
        ae.fit(x_train, x_train, epochs=ee, batch_size=bb, shuffle=True, verbose=1,validation_data=(x_test, x_test), callbacks=[TensorBoard(log_dir=log_dir)])



### Test Loss function with 40batch size and 100 epochs



In [ ]:
from keras.callbacks import TensorBoard
import tensorflow as tf
from datetime import datetime
%load_ext tensorboard


tf.random.set_seed(7)


loss_func = ['mean_squared_error', 'binary_crossentropy']

for ll in loss_func:

    log_dir = './grid-search/logs/find_loss_function_with_tests/' + str(ll)
    print('logs save in:', log_dir)

    work_text = str(ll) + '   LOSS Function'
    work_time = datetime.now().strftime("%Y%m%d-%H%M%S")
    work_config = '40batch_sizes /// 100epochs /// binary_accuracy /// Adam@default'
    dataset_config = 'train:' + str(x_train.shape) + ';;; test:' + str(x_test.shape)
    fw =  tf.summary.create_file_writer(log_dir)
    with fw.as_default():
        tf.summary.text("Test Title", work_text, step=0)
        tf.summary.text("Time", work_time, step=0)
        tf.summary.text("Config", work_config, step=0)
        tf.summary.text("Datasets_config", dataset_config, step=0)

    ae.compile(optimizer='adam', loss=ll, metrics=['accuracy'])
    ae.fit(x_train, x_train, epochs=100, batch_size=40, shuffle=True, verbose=1,validation_data=(x_test, x_test), callbacks=[TensorBoard(log_dir=log_dir)])



### Add loss and optimzer 


mse loss function and adam-series optimzer get lowest result...

In [ ]:
from keras.callbacks import TensorBoard
import tensorflow as tf
from datetime import datetime
%load_ext tensorboard


tf.random.set_seed(7)


# loss_func = ['mean_squared_error', 'binary_crossentropy']
# opt = ['SGD', 'RMSprop', 'Adagrad', 'Adadelta', 'Adam', 'Adamax', 'Nadam']

loss_func = ['binary_crossentropy']
opt = ['Adadelta', 'Adam', 'Adamax', 'Nadam']

for ll in loss_func:
    for op in opt:

        log_dir = './grid-search/logs/find_loss_function_and_optimizers_with_tests/' + str(ll) +'_' +str(op)
        print('logs save in:', log_dir)

        work_text = str(ll) + ' &&& ' +str(op)
        work_time = datetime.now().strftime("%Y%m%d-%H%M%S")
        work_config = '40batch_sizes /// 100epochs /// accuracy'
        dataset_config = 'train:' + str(x_train.shape) + ';;; test:' + str(x_test.shape)
        fw =  tf.summary.create_file_writer(log_dir)
        with fw.as_default():
            tf.summary.text("Test Title", work_text, step=0)
            tf.summary.text("Time", work_time, step=0)
            tf.summary.text("Config", work_config, step=0)
            tf.summary.text("Datasets_config", dataset_config, step=0)

        ae.compile(optimizer=op, loss=ll, metrics=['accuracy'])
        ae.fit(x_train, x_train, epochs=100, batch_size=40, shuffle=True, verbose=1,validation_data=(x_test, x_test), callbacks=[TensorBoard(log_dir=log_dir)])